In [1]:
import sys

sys.path.append('/mnt/sdceph/users/nroy/radmc3d-2.0/python/radmc3dPy')
sys.path.append('/mnt/home/nroy/test/chimes-IGNIS/chimes-driver/ext-lib/pfh_python')

#from simpleplot import *
#from radmc3dPy import image
#from gizmopy.load_from_snapshot import load_from_snapshot
#from gizmopy.load_fire_snap import load_fire_snap
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['axes.facecolor']='w'


params = {"text.color" : 'w',
          "ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
'''
params = {"text.color" : 'k',
          "ytick.color" : "k",
          "xtick.color" : "k",
          "axes.labelcolor" : "k",
          "axes.edgecolor" : "k"}

'''
plt.rcParams.update(params)

In [2]:
#creating line_list, snapshot_list to be looped through and appended to file path to create arrays
line_list = ['CII_158mu', 'NII_6548A', 'OI_6300A', 'OI_63mu','OI_145mu', 'OIII_5007A', 'OIII_88mu', 'OIV_25mu', 'NeII_12mu', 'NeIII_15mu', 'NeIII_3869A', 'NeV_14mu', 'NeV_3426A', 'NeVI_7mu',  'SII_6716A', 'SII_6731A', 'SIII_18mu', 'CO_J10']
snapshot_list = ['151', '152', '155']
#file.close()

In [3]:
for s in snapshot_list:
    MOMENTS = s + "_moments_all_lines.hdf5"
    file = h5.File(MOMENTS, 'w')
    for l in line_list:
        parent = '/home/jovyan/radmc3d_data/subtract_com_velocity/A4_33000_snum' + s + '/base_resolution_128/' + l + '/' 
        filepath_tot = parent + 'image_tot.hdf5'
        filepath_con = parent + 'image_continuum.hdf5'
        file_con = h5.File(filepath_con,'r')
        file_tot = h5.File(filepath_tot,'r')
        
        npix_x = np.shape(file_tot['image_array'][:])[0]
        npix_y = np.shape(file_tot['image_array'][:])[1]

        image_array = file_tot['image_array'][:] - file_con['image_array'][:]

        image_array[image_array < 0] = 0

        frequency_array = 3e8 / (file_tot['lambda_array'][:] * 1e-6)

        #moment 0
        integrated_image_array_moment0 = np.zeros((npix_x, npix_y))
        delta_nu = np.abs(frequency_array[1:] - frequency_array[:-1])
        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment0[i, j] = np.sum(delta_nu * (image_array[i, j, :-1] + image_array[i, j, 1:]) / 2.0)

        integrated_image_array_moment0[0,0] = 0 #final moment 0 array
        moment0 = np.asarray(integrated_image_array_moment0) #final moment 0 array - to go in hdf5
        file[l+"/M0"] = moment0 #creates M0 attribute for corresponding array
        
        #moment 1
        #N = np.size(frequency_array)
        lambda_0_dict = {"CII_158mu": 157.62730407714844,
                 "NII_6548A": 0.6545339822769165,
                 "OI_6300A": 0.62976968288421631,
                 "OI_63mu": 63.141555786132812,
                 "OI_145mu": 145.43453979492188,
                 "OIII_5007A": 0.50047838687896729,
                 "OIII_88mu": 88.295417785644531,
                 "OIV_25mu": 25.87542724609375,
                 "NeII_12mu": 12.804655075073242,
                 "NeIII_15mu": 15.543784141540527,
                 "NeIII_3869A" : 0.3867171793754335,
                 "NeV_14mu": 14.316787719726562,
                 "NeV_3426A" : 0.34246401541369015,
                 "NeVI_7mu": 7.6469192504882812,
                 "SII_6716A": 0.67136573791503906,
                 "SII_6731A": 0.67280274629592896,
                 "SIII_18mu": 18.69268798828125,
                 "CO_J10": 2600.757634}
                    
        lambda_0 = lambda_0_dict[l]
        lambda_array = np.array(file_con['lambda_array'])
        velocity_arr = (lambda_array - lambda_0) * 3e5 / lambda_array 
        
        
        moment1_noint = velocity_arr * image_array

        integrated_image_array_moment1 = np.zeros((npix_x, npix_y))

        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment1[i, j] = np.sum(delta_nu * (moment1_noint[i, j, :-1] + moment1_noint[i, j, 1:]) / 2.0)

        #integrated_image_array_moment1[0,0] = 0

        isolate_moment_1 = integrated_image_array_moment1 / integrated_image_array_moment0
        #isolate_moment_1[0,0] = 0 #final moment 1 array
        moment1 = np.asarray(isolate_moment_1) #final moment 1 array - to go in hdf5
        file[l+"/M1"] = moment1 #creates M1 attribute for corresponding array
        
        #moment 2
        isolate_moment_1_3D = np.empty([len(image_array), len(image_array), len(velocity_arr)])

        for i in range(len(velocity_arr)):
            for j in range(len(image_array)):
                for k in range(len(image_array)):
                    isolate_moment_1_3D[j][k][i] = isolate_moment_1[j][k]

        moment2_noint = image_array * ((velocity_arr - isolate_moment_1_3D) * (velocity_arr - isolate_moment_1_3D))

        integrated_image_array_moment2 = np.zeros((npix_x, npix_y))

        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment2[i, j] = np.sum(delta_nu * (moment2_noint[i, j, :-1] + moment2_noint[i, j, 1:]) / 2.0)

        integrated_image_array_moment2[0,0] = 0

        isolate_moment_2 = integrated_image_array_moment2 / integrated_image_array_moment0
        isolate_moment_2[0,0] = 0

        isolate_moment_2 = np.sqrt(isolate_moment_2) #final moment 2 array
        moment2 = np.asarray(isolate_moment_2) #final moment 2 array - to go in hdf5
        file[l+"/M2"] = moment2 #creates M2 attribute for corresponding array
    file.close()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in true_divide
